In [1]:
# imports
import glob
import pathlib
import numpy as np
import tensorflow as tf
from matplotlib import pyplot
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# # authenticate user
# auth.authenticate_user()
# print('Authenticated')

## The Discriminator

In [3]:
# def define_discriminator(in_shape=(32,32,3)):
# 	model = Sequential()
# 	# normal
# 	model.add(Conv2D(64, (3,3), padding='same', input_shape=in_shape))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# downsample
# 	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# downsample
# 	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# downsample
# 	model.add(Conv2D(256, (3,3), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# classifier
# 	model.add(Flatten())
# 	model.add(Dropout(0.4))
# 	model.add(Dense(1, activation='sigmoid'))
# 	# compile model
# 	opt = Adam(lr=0.0002, beta_1=0.5)
# 	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
# 	return model

In [4]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,3), n_classes=13):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    # downsample to 14x14
    # fe = Conv2D(32, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(in_image)
    fe = Conv2D(32, (3,3), padding='same', kernel_initializer=init)(in_image)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # normal
    fe = Conv2D(64, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # downsample to 7x7
    fe = Conv2D(128, (3,3), strides=(2,2), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # normal
    fe = Conv2D(256, (3,3), padding='same', kernel_initializer=init)(fe)
    fe = BatchNormalization()(fe)
    fe = LeakyReLU(alpha=0.2)(fe)
    fe = Dropout(0.5)(fe)
    # flatten feature maps
    fe = Flatten()(fe)
    # real/fake output
    out1 = Dense(1, activation='sigmoid')(fe)
    # class label output
    out2 = Dense(n_classes, activation='softmax')(fe)
    # define model
    model = Model(in_image, [out1, out2])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [5]:
# define the discriminator model
model = define_discriminator()

In [6]:
# summarize the model
# model.summary()

In [7]:
# plot the model
# plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=True)

## The Generator

In [8]:
# def define_generator(latent_dim):
# 	model = Sequential()
# 	# foundation for 4x4 image
# 	n_nodes = 256 * 4 * 4
# 	model.add(Dense(n_nodes, input_dim=latent_dim))
# 	model.add(LeakyReLU(alpha=0.2))
# 	model.add(Reshape((4, 4, 256)))
# 	# upsample to 8x8
# 	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# upsample to 16x16
# 	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# upsample to 32x32
# 	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
# 	model.add(LeakyReLU(alpha=0.2))
# 	# output layer
# 	model.add(Conv2D(3, (3,3), activation='tanh', padding='same'))
# 	return model

In [9]:
# define the standalone generator model
def define_generator(latent_dim, n_classes=13):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # label input
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # linear multiplication
    n_nodes = 7 * 7
    li = Dense(n_nodes, kernel_initializer=init)(li)
    # reshape to additional channel
    li = Reshape((7, 7, 1))(li)
    # image generator input
    in_lat = Input(shape=(latent_dim,))
    # foundation for 7x7 image
    n_nodes = 384 * 7 * 7
    gen = Dense(n_nodes, kernel_initializer=init)(in_lat)
    gen = Activation('relu')(gen)
    gen = Reshape((7, 7, 384))(gen)
    # merge image gen and label input
    merge = Concatenate()([gen, li])
    # upsample to 14x14
    gen = Conv2DTranspose(192, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(merge)
    gen = BatchNormalization()(gen)
    gen = Activation('relu')(gen)
    # upsample to 28x28
    gen = Conv2DTranspose(3, (5,5), strides=(2,2), padding='same', kernel_initializer=init)(gen)
    out_layer = Activation('tanh')(gen)
    # define model
    model = Model([in_lat, in_label], out_layer)
    return model

In [10]:
# define the size of the latent space
latent_dim = 100

In [11]:
# define the generator model
g_model = define_generator(latent_dim)

In [12]:
# summarize the model
# g_model.summary()

In [13]:
# plot the model
# plot_model(g_model, to_file='generator_plot.png', show_shapes=True, show_layer_names=True)

## Combined Discriminator and Generator

In [14]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect the outputs of the generator to the inputs of the discriminator
    gan_output = d_model(g_model.output)
    # define gan model as taking noise and label and outputting real/fake and label outputs
    model = Model(g_model.input, gan_output)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], optimizer=opt)
    return model

In [15]:
# !rm -rf assets_april_2nd

In [16]:
# download prepared dataset from GCS
# !gsutil -m cp -r gs://trisha-ai-in-dermatology/train .
# !gsutil cp -r gs://ai-in-dermatology/data/train_r .
# !gsutil cp -r gs://ai-in-dermatology/data/test_r .

In [17]:
CLASS_NAMES = np.array([item.name for item in pathlib.Path('train').glob('*')])
CLASS_NAMES

array(['seborrhoeic-dermatitis-eczema', 'atopic-dermatitis-eczema',
       'psoriasis', 'lichen-planus', 'mycosis-fungoides-mf-nodular',
       'mycosis-fungoides-mf-patch', 'vitiligo', 'granuloma-annulare',
       'pityriasis-versicolor', 'pityriasis-rosea', 'scabies',
       'mycosis-fungoides-mf-plaque',
       'kaposi-sarcoma-patch-plaque-nodule'], dtype='<U34')

In [18]:
import os
cpt = sum([len(files) for r, d, files in os.walk('train')])
cpt

1592

In [19]:
BATCH_SIZE = 32
IMG_HEIGHT = 28 #224
IMG_WIDTH = 28 #224
CLASS_SIZE = len(CLASS_NAMES)
# STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [20]:
def make_generator():
    train_datagen = ImageDataGenerator()#(featurewise_center=True,
                                      #featurewise_std_normalization=True)
    train_generator = train_datagen.flow_from_directory(
        'train',
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=cpt,
        class_mode='binary',
        classes = list(CLASS_NAMES))
    return train_generator

In [21]:
# images_ds = (tf.data.Dataset.from_generator(make_generator, (tf.float32, tf.float32)).unbatch()).cache().repeat()

In [22]:
# load images
def load_real_samples():
    # load dataset
    images_ds = make_generator()
    (trainX, trainy) = next(images_ds)
    # (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    # X = expand_dims(trainX, axis=-1)
    X = trainX
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    print(X.shape, trainy.shape)
    return [X, trainy]

In [23]:
# select real samples
def generate_real_samples(dataset, n_samples):
    # split into images and labels
    images, labels = dataset
    # choose random instances
    ix = randint(0, images.shape[0], n_samples)
    # select images and labels
    X, labels = images[ix], labels[ix]
    # generate class labels
    y = ones((n_samples, 1))
    return [X, labels], y

In [24]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples, n_classes=13):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    z_input = x_input.reshape(n_samples, latent_dim)
    # generate labels
    labels = randint(0, n_classes, n_samples)
    return [z_input, labels]

In [25]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    # generate points in latent space
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([z_input, labels_input])
    # create class labels
    y = zeros((n_samples, 1))
    return [images, labels_input], y

In [26]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, latent_dim, n_samples=100, assets_dir='assets_april_3rd'):
    if not os.path.isdir(assets_dir):
        os.makedirs(assets_dir)
      
    # prepare fake examples
    [X, _], _ = generate_fake_samples(g_model, latent_dim, n_samples)
    # scale from [-1,1] to [0,1]
    X = (X + 1) / 2.0
    # plot images
    pyplot.figure(figsize=(20,20))
    for i in range(100):
        # define subplot
        pyplot.subplot(10, 10, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(X[i, :, :, 0])#, cmap='gray_r')
    # save plot to file
    filename1 = '%s/generated_plot_%04d.png' % (assets_dir, step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = '%s/model_%04d.h5' % (assets_dir, step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))
    

    
#  def _show_image_grid(image_grid):
#   plt.figure(figsize=(18,18))
#   plt.gca().set_axis_off()
#   plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
#               hspace = 0, wspace = 0)
#   plt.margins(0,0)
#   plt.gca().xaxis.set_major_locator(plt.NullLocator())
#   plt.gca().yaxis.set_major_locator(plt.NullLocator())
#   plt.imshow(np.squeeze(image_grid).astype(np.uint8))
#   plt.show()


# create and save a plot of generated images
# def save_plot(examples, n_examples):
# 	# plot images
# 	for i in range(n_examples):
# 		# define subplot
# 		pyplot.subplot(sqrt(n_examples), sqrt(n_examples), 1 + i)
# 		# turn off axis
# 		pyplot.axis('off')
# 		# plot raw pixel data
# 		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
# 	pyplot.show()

In [27]:
# ! rm generated_*
# ! rm model_*

In [28]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=1000, n_batch=64):
    # calculate the number of batches per training epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # calculate the size of half a batch of samples
    half_batch = int(n_batch / 2)
    # manually enumerate epochs
    for i in range(n_steps):
        # get randomly selected 'real' samples
        [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
        # update discriminator model weights
        _,d_r1,d_r2 = d_model.train_on_batch(X_real, [y_real, labels_real])
        # generate 'fake' examples
        [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
        # update discriminator model weights
        _,d_f,d_f2 = d_model.train_on_batch(X_fake, [y_fake, labels_fake])
        # prepare points in latent space as input for the generator
        [z_input, z_labels] = generate_latent_points(latent_dim, n_batch)
        # create inverted labels for the fake samples
        y_gan = ones((n_batch, 1))
        # update the generator via the discriminator's error
        _,g_1,g_2 = gan_model.train_on_batch([z_input, z_labels], [y_gan, z_labels])
        # summarize loss on this batch
        print('>%d, dr[%.3f,%.3f], df[%.3f,%.3f], g[%.3f,%.3f]' % (i+1, d_r1,d_r2, d_f,d_f2, g_1,g_2))
        # evaluate the model performance every 'epoch'
        if (i+1) % (bat_per_epo * 100) == 0:
            print('Summarizing performance......')
            summarize_performance(i, g_model, latent_dim)

In [29]:
# size of the latent space
latent_dim = 100

In [30]:
# create the discriminator
discriminator = define_discriminator()

In [31]:
# create the generator
generator = define_generator(latent_dim)

In [32]:
# create the gan
gan_model = define_gan(generator, discriminator)

In [33]:
# load image data
dataset = load_real_samples()

Found 1588 images belonging to 13 classes.
(1588, 28, 28, 3) (1588,)


In [34]:
# train model
train(generator, discriminator, gan_model, dataset, latent_dim)

>1, dr[1.251,3.658], df[2.182,4.051], g[0.692,2.564]
>2, dr[0.485,4.178], df[1.094,3.510], g[0.697,2.564]
>3, dr[1.204,3.906], df[0.771,3.761], g[0.697,2.568]
>4, dr[1.055,3.641], df[0.677,3.511], g[0.699,2.566]
>5, dr[0.924,3.503], df[0.850,3.826], g[0.699,2.564]
>6, dr[0.680,3.552], df[0.679,3.879], g[0.700,2.566]
>7, dr[0.633,3.280], df[0.805,3.970], g[0.705,2.569]
>8, dr[0.650,3.032], df[0.397,3.923], g[0.704,2.566]
>9, dr[0.780,3.403], df[0.819,3.766], g[0.708,2.560]
>10, dr[0.663,2.997], df[0.832,3.918], g[0.710,2.566]
>11, dr[0.928,3.817], df[0.601,4.073], g[0.710,2.579]
>12, dr[0.586,2.921], df[0.494,3.410], g[0.711,2.568]
>13, dr[0.605,3.278], df[0.567,3.440], g[0.717,2.570]
>14, dr[0.349,3.252], df[0.324,3.429], g[0.725,2.579]
>15, dr[0.511,3.124], df[0.289,3.635], g[0.718,2.548]
>16, dr[0.465,3.239], df[0.446,4.140], g[0.713,2.579]
>17, dr[0.509,3.399], df[0.341,3.626], g[0.721,2.591]
>18, dr[0.449,2.945], df[0.467,3.434], g[0.716,2.579]
>19, dr[0.570,3.584], df[0.545,3.935]